## TLDR

This training is based on Tensorflow Object Detection API and Tensorflow version 2. Before start training, API installation is needed, link to the official [github](https://github.com/tensorflow/models/tree/master/research/object_detection) to find the installation guide. 

After API installed, please prepare pre-train model and training data. You can find pre-train model on official [model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) or third party model. About training data, we generated it on [roboflow](https://roboflow.com/) you can also use other method to generate it.

## Setup paths

In [ ]:
MODEL_DIR = 'training_model' # file dir where you create
PIPILINE = MODEL_DIR + '/pipeline.config'
TRAINED_CHECKPOINT = 'train_0620_V10'
TFLITE_MODEL_DIR = TRAINED_CHECKPOINT + '/tflite'

## Check python version

In [15]:
PYTHON_PATH = '/home/mlst01/.conda/envs/conda-py36env/bin/python'
!{PYTHON_PATH} --version

Python 3.6.10 :: Anaconda, Inc.


## Training

In [30]:
!cat {PIPILINE}

model {
  ssd {
    num_classes: 1
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.9700000286102295
          center: true
          scale: true
          epsilon: 0.0010000000474974513
          train: true
        }
      }
      override_base_feature_extractor_hyperparams: true
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
    

In [17]:
# after saved run.sh run this line

!{PYTHON_PATH} model_main_tf2.py \
    --pipeline_config_path={PIPILINE} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr

600 per-step time 0.144s loss=0.169
INFO:tensorflow:Step 37700 per-step time 0.135s loss=0.192
I0624 23:57:12.448374 140551522027328 model_lib_v2.py:685] Step 37700 per-step time 0.135s loss=0.192
INFO:tensorflow:Step 37800 per-step time 0.132s loss=0.133
I0624 23:57:25.770785 140551522027328 model_lib_v2.py:685] Step 37800 per-step time 0.132s loss=0.133
INFO:tensorflow:Step 37900 per-step time 0.139s loss=0.154
I0624 23:57:39.378085 140551522027328 model_lib_v2.py:685] Step 37900 per-step time 0.139s loss=0.154
INFO:tensorflow:Step 38000 per-step time 0.143s loss=0.170
I0624 23:57:52.839313 140551522027328 model_lib_v2.py:685] Step 38000 per-step time 0.143s loss=0.170
INFO:tensorflow:Step 38100 per-step time 0.130s loss=0.199
I0624 23:58:06.658087 140551522027328 model_lib_v2.py:685] Step 38100 per-step time 0.130s loss=0.199
INFO:tensorflow:Step 38200 per-step time 0.143s loss=0.211
I0624 23:58:20.082746 140551522027328 model_lib_v2.py:685] Step 38200 per-step time 0.143s loss=0.21

## Visualize in tensorboard

In [32]:
!/home/mlst01/.conda/envs/conda-py36env/bin/tensorboard \
    --logdir={TRAINED_CHECKPOINT}

2021-06-26 00:49:45.806339: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6007/ (Press CTRL+C to quit)
^C


## Evaluation

In [14]:
!{PYTHON_PATH} model_main_tf2.py \
    --pipeline_config_path={PIPILINE} \
    --model_dir={MODEL_DIR} \
    --checkpoint_dir={TRAINED_CHECKPOINT} \
    --alsologtostderr

ices: 0
W0624 16:01:01.355125 140660113827648 model_lib_v2.py:1066] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I0624 16:01:01.355263 140660113827648 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0624 16:01:01.355327 140660113827648 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0624 16:01:01.355396 140660113827648 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0624 16:01:01.355475 140660113827648 model_lib_v2.py:1087] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2021-06-24 16:01:01.362598: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU inst

## Freeze to tflite

In [24]:
print(TFLITE_MODEL_DIR)

/home/mlst01/CarCarder/tensorflow_object_detection_ssdmobilenet/train_0620_V10/tflite


In [25]:
!{PYTHON_PATH} export_tflite_graph_tf2.py \
    --pipeline_config_path {PIPILINE} \
    --trained_checkpoint_dir {MODEL_DIR} \
    --output_directory {TFLITE_MODEL_DIR}

image_features/2/1'))], False), {}).
I0625 00:32:39.181068 139926403286848 def_function.py:1170] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4017ed86d8>, TensorSpec(shape=(None, 40, 40, 32), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4017ed8a90>, TensorSpec(shape=(None, 20, 20, 96), dtype=tf.float32, name='image_features/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4017ed8978>, TensorSpec(shape=(None, 10, 10, 1280), dtype=tf.float32, name='image_features/2/1'))], False), {}).
INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4017f0f940>, TensorSpec(shape=(None, 40, 40, 32), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f4017f0f4a8>, TensorSpec(shape=(None, 20,

In [26]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(f'{TFLITE_MODEL_DIR}/saved_model') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open(f'{TFLITE_MODEL_DIR}/tflite_model.tflite', 'wb') as f:
  f.write(tflite_model)

ModuleNotFoundError: No module named 'google'

## Freeze to pretrained model

Go to `material` dir open up `freeze_graph.sh` and change the arguments

In [ ]:
# after saved to_tflite.sh run this line

!source material/freeze_graph.sh